## CSC 580: Critical Thinking 2 - Predicting Future Sales
In a nutshell, *sales_data_test.csv* and *sales_data_test.csv* contain data that will be used to train a neural network to predict how much money can be expected form the future sale of new video games. The .csv files were retrieved from one of [Toni Esteves repos](https://github.com/toniesteves/adam-geitgey-building-deep-learning-keras/tree/master/03). 

The columns in the data are defined as follows:
- critic_rating : an average rating out of five stars
- is_action : tells us if this was an action game
- is_exclusive_to_us : tells us if we have an exclusiv deal to sell this game
- is_portable : tells us if this game runs on a handheld video game system
- is_role_playing : tells us if this is a role-playing game
- is_sequel : tells us if this game was a sequel to an earlier video game and part of an ongoing series
- is_sports : tells us if this was a sports game
- suitable_for_kids : tells us if this game is appropriate for all ages
- total_earning : tells us how much money the store has earned in total from selling the game to all customers
- unit_price : tells us for how much a single copy of the game retailed

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras_tuner as kt
from keras.models import Sequential
from keras import layers 
from keras import activations
from keras import losses

#### Step 1: Prepare the Dataset
The numerical data needs to be scaled for better network training

In [2]:
# Load the training and testing data
train_data = pd.read_csv("sales_data_training.csv", dtype=float)
test_data = pd.read_csv("sales_data_test.csv", dtype = float)

# Scale the data using sklearn
scaler = MinMaxScaler(feature_range=(0,1))
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.fit_transform(test_data)

# Print out adjustment
print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

# Create new DataFrames
df_train_scaled = pd.DataFrame(train_data_scaled, columns=train_data.columns.values)
df_test_scaled = pd.DataFrame(test_data_scaled, columns=test_data.columns.values)

# Save scaled data
df_train_scaled.to_csv("sales_data_training_scaled.csv", index=False)
df_test_scaled.to_csv("sales_data_testing_scaled.csv", index=False)


Note: total_earnings values were scaled by multiplying by 0.0000042367 and adding -0.153415


#### Part 2: Coding the Network

In [3]:
# Load the training data
training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Splitting some off for validation set for HPO
X_TRAIN, X_VAL, Y_TRAIN, Y_VAL = train_test_split(X, Y, train_size=0.8)

In [4]:
# Model definition
def build_model(hp):
    model = Sequential()
    model.add(layers.Input((9,))) # Input

    # Tune the number of layers
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Tune number of units separately
                units=hp.Int("units_{i}", min_value=32, max_value=512, step=32),
                activation=activations.relu
            )
        )
    model.add(layers.Dense(1, activation=activations.linear)) # Output

    # Tune the learning rate
    learning_rate = hp.Float("lr", min_value = 1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
        loss="mse", 
        metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.RootMeanSquaredError()])

    return model

In [5]:
# Tuning the model's hyperparameters
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective=kt.Objective("val_root_mean_squared_error", direction="min"),
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory="hpo",
    project_name="sales_predictions"
)

# Summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_{i} (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


2022-02-23 15:13:12.735598: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Part 3: Training the Network

In [6]:
# Search for the best hyperparemeter configurations
tuner.search(X_TRAIN, Y_TRAIN, epochs=10, batch_size=25, validation_data=(X_VAL, Y_VAL))

Trial 10 Complete [00h 00m 03s]
val_root_mean_squared_error: 0.013163530733436346

Best val_root_mean_squared_error So Far: 0.005665629170835018
Total elapsed time: 00h 00m 35s
INFO:tensorflow:Oracle triggered exit


In [7]:
tuner.results_summary()

Results summary
Results in hpo/sales_predictions
Showing 10 best trials
Objective(name='val_root_mean_squared_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 3
units_{i}: 480
lr: 0.0004542815732661726
Score: 0.005665629170835018
Trial summary
Hyperparameters:
num_layers: 3
units_{i}: 480
lr: 0.0006432059239436091
Score: 0.006807776866480708
Trial summary
Hyperparameters:
num_layers: 3
units_{i}: 288
lr: 0.0006382243288146399
Score: 0.007407315541058779
Trial summary
Hyperparameters:
num_layers: 2
units_{i}: 384
lr: 0.00602482242977811
Score: 0.007965064141899347
Trial summary
Hyperparameters:
num_layers: 2
units_{i}: 128
lr: 0.0034818688017796847
Score: 0.008347525028511882
Trial summary
Hyperparameters:
num_layers: 1
units_{i}: 224
lr: 0.0016046549825965487
Score: 0.011732041370123625
Trial summary
Hyperparameters:
num_layers: 2
units_{i}: 224
lr: 0.0002140821791690464
Score: 0.013163530733436346
Trial summary
Hyperparameters:
num_layers: 1
units_{i}: 288
lr: 0.000

In [8]:
# Get the top 2 models
models = tuner.get_best_models(num_models=2)
best_model = models[0]

best_model.build()
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 480)               4800      
_________________________________________________________________
dense_1 (Dense)              (None, 480)               230880    
_________________________________________________________________
dense_2 (Dense)              (None, 480)               230880    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 481       
Total params: 467,041
Trainable params: 467,041
Non-trainable params: 0
_________________________________________________________________


In [9]:
best_model.fit(X,Y,batch_size = 100, epochs = 50, verbose=2, shuffle=True)

Epoch 1/50
10/10 - 0s - loss: 1.8791e-05 - mean_squared_error: 1.8791e-05 - root_mean_squared_error: 0.0043
Epoch 2/50
10/10 - 0s - loss: 1.6397e-05 - mean_squared_error: 1.6397e-05 - root_mean_squared_error: 0.0040
Epoch 3/50
10/10 - 0s - loss: 1.3992e-05 - mean_squared_error: 1.3992e-05 - root_mean_squared_error: 0.0037
Epoch 4/50
10/10 - 0s - loss: 1.2642e-05 - mean_squared_error: 1.2642e-05 - root_mean_squared_error: 0.0036
Epoch 5/50
10/10 - 0s - loss: 1.4944e-05 - mean_squared_error: 1.4944e-05 - root_mean_squared_error: 0.0039
Epoch 6/50
10/10 - 0s - loss: 1.5769e-05 - mean_squared_error: 1.5769e-05 - root_mean_squared_error: 0.0040
Epoch 7/50
10/10 - 0s - loss: 1.2750e-05 - mean_squared_error: 1.2750e-05 - root_mean_squared_error: 0.0036
Epoch 8/50
10/10 - 0s - loss: 1.1694e-05 - mean_squared_error: 1.1694e-05 - root_mean_squared_error: 0.0034
Epoch 9/50
10/10 - 0s - loss: 1.1211e-05 - mean_squared_error: 1.1211e-05 - root_mean_squared_error: 0.0033
Epoch 10/50
10/10 - 0s - los

#### Part 4 : Evaluating and Saving the Network

In [10]:
testing_data_df = pd.read_csv("sales_data_testing_scaled.csv")

X_TEST = testing_data_df.drop('total_earnings', axis=1).values
Y_TEST = testing_data_df[['total_earnings']].values

In [12]:
test_error_rate = best_model.evaluate(X_TEST, Y_TEST, verbose=2)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate[0]))

13/13 - 0s - loss: 0.0011 - mean_squared_error: 0.0011 - root_mean_squared_error: 0.0327
The mean squared error (MSE) for the test data set is: 0.0010661539854481816


In [13]:
best_model.save("trained_model_hpo.h5")
print("Model saved to disk")

Model saved to disk


In [15]:
model = tf.keras.models.load_model("trained_model.h5")
hp_model = tf.keras.models.load_model("trained_model_hpo.h5")

test_error_rate = model.evaluate(X_TEST, Y_TEST, verbose=2)
hp_test_error_rate = (hp_model.evaluate(X_TEST, Y_TEST, verbose=2))[0]
print("The mean squared error (MSE) for the test data set is (No-HPO): {}".format(test_error_rate))
print("The mean squared error (MSE) for the test data set is (HPO): {}".format(hp_test_error_rate))

13/13 - 0s - loss: 0.0015
13/13 - 0s - loss: 0.0011 - mean_squared_error: 0.0011 - root_mean_squared_error: 0.0327
The mean squared error (MSE) for the test data set is (No-HPO): 0.0014870389131829143
The mean squared error (MSE) for the test data set is (HPO): 0.0010661539854481816


#### Part 5: Making Predictions

In [19]:
X_PRED = pd.read_csv("proposed_new_product.csv").values
pred = model.predict(X_PRED)
pred_hp = hp_model.predict(X_PRED)

# Scale
pred = pred[0][0]
pred_hp = pred_hp[0][0]
pred = (pred + scaler.min_[8]) / scaler.scale_[8]
pred_hp = (pred_hp + scaler.min_[8]) / scaler.scale_[8]

print("Earnings Prediction for Proposed Product - (Non-HP): ${}  |  (HP): ${}".format(pred, pred_hp))

Earnings Prediction for Proposed Product - (Non-HP): $168958.729593277  |  (HP): $168679.31187504533
